In [31]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file, to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import numpy as np
import random
import sys
import io

In [32]:
strategy  = tf.distribute.get_strategy()
with strategy.scope():
    print(f"{tf.config.list_physical_devices('GPU') }")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [33]:
data_file_path = '/kaggle/input/kieu-story-txt/truyen_kieu_data_pre.txt'

def load_data() -> tuple:
    rs: map = {}
    ls = []
    tmp_line = ""
    with open(file=data_file_path, mode="rt") as i_f:
        count = 0
        for line in i_f:
            line = line.strip()
            if "" == line: continue
            if count % 2 == 0 and tmp_line!="":
                ls.append(tmp_line.strip())
                tmp_line = ""
                pass
#             tmp_line += (" | " if tmp_line!="" else "") + line
            tmp_line += " "+line
            a_word = line.split()
            for word in a_word:
                if word not in rs:
                    rs[word] = 0
                    pass
                rs[word]+=1
                pass
            pass
            count+=1
        pass
    return rs, ls

In [34]:
bag_of_words, coupled_lines = load_data()
# bag_of_words["|"] = len(coupled_lines)
# bag_of_words["\\"] = len(coupled_lines) - 1
# ix_to_word = [*bag_of_words.keys()]
keys = [*bag_of_words.keys()]
word_to_ix = { w:i for i,w in enumerate(keys) }
ix_to_word = { i:w for i,w in enumerate(keys) }
# coupled_lines
# values = [*bag_of_word.values()]
# sum(values)
# len(bag_of_words)
# ix_to_word
X_train = [[word_to_ix[word] for word in coupled_line.split()] for coupled_line in coupled_lines]

delta_line = 1
# y_train = [line[5] for line in X_train[delta_line:]]
y_train = [*X_train[delta_line:]]
y_train = [e[0] for e in y_train]
for c in range(delta_line):
    y_train.append(X_train[c][0])
    pass

total_word = len(bag_of_words)
max_input_len = 14
X_train = np.asarray(a=X_train, dtype=int)
y_train = np.asarray(y_train) #.reshape(X_train.shape[0], 1)
# print(y_train.shape)
y_train = to_categorical(y_train, num_classes=total_word, dtype=int)
# X_train.shape
# y_train.shape
# len(coupled_lines[0].split())
# coupled_lines

In [51]:
model: tf.keras.Model = tf.keras.Sequential(name="LSTM-RNN")
# input = tf.keras.layers.Input(shape=(15))
embedding_1 = tf.keras.layers.Embedding(input_dim=total_word, output_dim=2048, input_length=max_input_len)
bidrect_1 = tf.keras.layers.Bidirectional(\
    layer=tf.keras.layers.LSTM(units=14*32, return_sequences=True, go_backwards=False),\
    backward_layer=tf.keras.layers.LSTM(units=14*24, return_sequences=True, go_backwards=True))
dropout = tf.keras.layers.Dropout(rate=0.4)
bidrect_2 = tf.keras.layers.Bidirectional(layer=tf.keras.layers.LSTM(units=14*16, return_sequences=True))
dropout = tf.keras.layers.Dropout(rate=0.6)
lstm = tf.keras.layers.LSTM(units=14*16)
output_1 = tf.keras.layers.Dense(units=total_word, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-2))
output_2 = tf.keras.layers.Dense(units=total_word, activation="softmax")

# model.add(input)
model.add(embedding_1)
model.add(bidrect_1)
model.add(bidrect_2)
model.add(dropout)
model.add(lstm)
model.add(output_1)
model.add(output_2)

model.summary()


Model: "LSTM-RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 14, 2048)          4900864   
                                                                 
 bidirectional_13 (Bidirecti  (None, 14, 784)          7680064   
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 14, 448)          1808128   
 onal)                                                           
                                                                 
 dropout_14 (Dropout)        (None, 14, 448)           0         
                                                                 
 lstm_34 (LSTM)              (None, 224)               603008    
                                                                 
 dense_18 (Dense)            (None, 2393)              538

In [52]:
tf.config.set_soft_device_placement(True) 
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# loss_fn = tf.keras.losses.sparse_categorical_crossentropy
loss_fn = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=loss_fn,
              metrics=["accuracy"])
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=loss_fn,
#               metrics=["accuracy"])

In [53]:
# strategy  = tf.distribute.get_strategy()
with strategy.scope():
    model.reset_metrics()
    model.reset_states()
# with tf.device('/cpu:0'):
# with tf.device("/gpu:0"):
    model.fit(x=X_train, y=y_train, epochs=160, batch_size=128, verbose=1)
#     model.fit(x=X_train, y=y_train, epochs=75, batch_size=64, verbose=1)
    pass

Epoch 1/160
13/13 [==============================] - 11s 171ms/step - loss: 10.3098 - accuracy: 0.0270
Epoch 2/160
13/13 [==============================] - 2s 113ms/step - loss: 7.6492 - accuracy: 0.0246
Epoch 3/160
13/13 [==============================] - 2s 135ms/step - loss: 6.7370 - accuracy: 0.0344
Epoch 4/160
13/13 [==============================] - 2s 124ms/step - loss: 6.3666 - accuracy: 0.0246
Epoch 5/160
13/13 [==============================] - 1s 78ms/step - loss: 6.1828 - accuracy: 0.0344
Epoch 6/160
13/13 [==============================] - 1s 89ms/step - loss: 6.0878 - accuracy: 0.0344
Epoch 7/160
13/13 [==============================] - 1s 68ms/step - loss: 6.0410 - accuracy: 0.0344
Epoch 8/160
13/13 [==============================] - 1s 89ms/step - loss: 6.0177 - accuracy: 0.0344
Epoch 9/160
13/13 [==============================] - 1s 77ms/step - loss: 6.0066 - accuracy: 0.0344
Epoch 10/160
13/13 [==============================] - 1s 67ms/step - loss: 5.9938 - accuracy: 

In [38]:
model.save('/kaggle/working/kieu_story')

Exception ignored in: <function _xla_gc_callback at 0x7eaa0a772320>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
import os
import subprocess
from IPython.display import FileLink, display
download_file_name = 'kieu_story'
path = f"/kaggle/working/{download_file_name}"
zip_name = f"/kaggle/working/{download_file_name}.zip"
command = f"zip {zip_name} {path} -r"
result = subprocess.run(command, shell=True, capture_output=True, text=True)
if result.returncode != 0:
    print("Unable to run zip command!")
    print(result.stderr)
display(FileLink(f'{download_file_name}.zip'))


In [54]:
import random as rd

num_of_word = max_input_len * 4
# seed = "đàn bà dễ có mấy tay đời xưa mấy mặt đời này mấy gan"
seed_arr = [ix_to_word[rd.randint(0, total_word)] for _ in range(14)]
seed_arr[11] = seed_arr[5]
seed = " ".join(seed_arr).strip()
seed_sequence = [word_to_ix[word] for word in seed.split()]
rs = ""
for i in range(num_of_word):
    if i % max_input_len == 0 and i != 0:
#         seed_sequence.append(" \\")
        rs += ("\n")
#         continue
    if i % max_input_len == 6:
#         seed_sequence.append(" | ")
        rs += ("\n")
#         continue
    prediction = model.predict(np.array([seed_sequence]), verbose="0")
    predicted_word_idx = np.argmax(prediction)
    predicted_word = ix_to_word[predicted_word_idx]
    
    seed_sequence.append(predicted_word_idx)
    
    if len(seed_sequence) % 6 == 0:
        seed_sequence = seed_sequence[6:]
    seed += " " + predicted_word
    rs += " " + predicted_word

print(rs)

 sinh thân đã cười nước dắt
 những này trong quyết một nhớ khen điều
 một xót dưới bóng vội hỏi
 phải đạo nhân khóc rằng cùng phải chén
 sầu nhớ dễ đường phòng nhặt
 giác thôi giác thôi giác thôi chàng chàng
 chàng chàng chàng chàng này tiếc
 lỡ lại nhớ dẫu thôi khi ví khi
